In [1]:
#Load required libraries
import pandas as pd

import tensorflow as tf
import keras
import numpy as np

from keras.models import Sequential, Model
from keras.layers import Activation, Dense
from keras.layers import Conv2D, GlobalAveragePooling2D, BatchNormalization
from keras.layers import Dense, Dropout
from keras.layers import Dense, Dropout, Activation, Flatten

In [87]:
# loading parameters
param_train = pd.read_csv('./parameters_BiSSE.txt', sep='\t', header=None, index_col=0, names=['lambda1', 'lambda2', 'turnover', 'sampling_frac', 'tree_size', 'mu1', 'mu2', 'net_rate1', 'net_rate2', 'q01', 'q10', 'lambda2_ratio', 'q01_ratio']).reset_index(drop=True)
param_test = pd.read_csv('./testset/parameters_BiSSE.txt', sep='\t', header=None, index_col=0, names=['lambda1', 'lambda2', 'turnover', 'sampling_frac', 'tree_size', 'mu1', 'mu2', 'net_rate1', 'net_rate2', 'q01', 'q10', 'lambda2_ratio', 'q01_ratio']).reset_index(drop=True)

# loading tree encoding
encoding = pd.read_csv('./Encoded_trees_BiSSE.csv', sep="\t", header=0, index_col=0).values.reshape(-1,1000,19)
encoding_test = pd.read_csv('./testset/Encoded_trees_BiSSE.csv', sep="\t", header=0, index_col=0).values.reshape(-1,1000,19)

In [88]:
#Reshape parameters (and rescale them) and encodings:

### TRAINING SET: PARAMETER VALUES
# extract rescaling factor (last line of each encoded tree)
param_train['resc_factor'] = encoding[:,-1,-1]

# rescale target values according to scaling factor
param_train['net_rat1_rescaled'] = param_train['net_rate1']*param_train['resc_factor']
param_train['net_rat2_rescaled'] = param_train['net_rate2']*param_train['resc_factor']
param_train['lambda1_rescaled'] = param_train['lambda1']*param_train['resc_factor']
param_train['lambda2_rescaled'] = param_train['lambda2']*param_train['resc_factor']
param_train['q01_rescaled'] = param_train['q01']*param_train['resc_factor']

### TESTING SET: PARAMETER VALUES
# extract rescaling factor  (last line of each encoded tree)
param_test['resc_factor'] = encoding_test[:,-1,-1]
# rescale target values
param_test['net_rat1_rescaled'] = param_test['net_rate1']*param_test['resc_factor']
param_test['net_rat2_rescaled'] = param_test['net_rate2']*param_test['resc_factor']
param_test['lambda1_rescaled'] = param_test['lambda1']*param_test['resc_factor']
param_test['lambda2_rescaled'] = param_test['lambda2']*param_test['resc_factor']
param_test['q01_rescaled'] = param_test['q01']*param_test['resc_factor']

# remove irrelevant columns: normalization factor
encoding=np.delete(encoding, -1, axis=1)
encoding_test=np.delete(encoding_test, -1, axis=1)

#Choice of the parameters to predict
target_1 = "turnover"
target_2 = "lambda1_rescaled"
target_3 = "lambda2_rescaled"
target_4 = "q01_rescaled"

#Add parameters to predict as labels for each simulation
Y = pd.DataFrame(param_train[[target_1, target_2, target_3, target_4]])
Y_test = pd.DataFrame(param_test[[target_1, target_2, target_3, target_4]])

#Fraction of the training trees to be used as validation and training set
valid_frac = 0.3
train_size_frac = 0.7

In [89]:
#Now insert an additional column with sampling proba for all nodes

samp_proba_list = np.array(param_train['sampling_frac'])
encoding=np.concatenate((encoding,np.repeat(samp_proba_list,999).reshape(-1,999,1)),axis=2)

samp_proba_list_test = np.array(param_test['sampling_frac'])
encoding_test=np.concatenate((encoding_test,np.repeat(samp_proba_list_test,999).reshape(-1,999,1)),axis=2)

In [90]:
# This function takes in the tree encodings for both training and testing datasets
# and processes them to have a uniform shape. It also pads the leaves and nodes 
# of the trees to ensure each tree has a fixed number of 500 leaves and nodes.

def encode_pad_0s_rootage(enc, enc_test):
    # Create an empty list to hold padded training encodings
    enc_pad = []
    
    # Iterate over each tree in the training dataset
    for i in range(enc.shape[0]):
        # Separate the leaves (where column 3 has value 1, which indicates leaves)
        leaves = enc[i][enc[i,:,3] == 1]
        # Sort leaves by their age (assumed to be in column 1)
        leaves = leaves[np.argsort(leaves[:, 1])]
        # Pad the leaves array with 0s until it has a maximum size of 500 leaves
        leaves = np.pad(leaves, [(0, (500 - leaves.shape[0])), (0, 0)], mode='constant')

        # Separate the nodes (where column 3 is greater than 1, indicating internal nodes)
        nodes = enc[i][enc[i,:,3] > 1]
        # Sort nodes by their age (assumed to be in column 1)
        nodes = nodes[np.argsort(nodes[:, 1])]
        # Copy the last node's value to balance the number of leaves and nodes
        nodes = np.append(nodes, nodes[-1].reshape(1, -1), axis=0)
        # Pad the nodes array with 0s to ensure a size of 500 nodes
        nodes = np.pad(nodes, [(0, (500 - nodes.shape[0])), (0, 0)], mode='constant')
        
        # Stack the leaves and nodes arrays together along axis 2 (creating 2 channels)
        enc_pad.append(np.stack((leaves, nodes), axis=2))
    
    # Now process the test dataset (same procedure as above)
    enc_pad_test = []
    for i in range(enc_test.shape[0]):
        # Extract and sort leaves
        leaves = enc_test[i][enc_test[i,:,3] == 1]
        leaves = leaves[np.argsort(leaves[:, 1])]
        # Pad leaves to ensure size of 500
        leaves = np.pad(leaves, [(0, (500 - leaves.shape[0])), (0, 0)], mode='constant')

        # Extract and sort nodes
        nodes = enc_test[i][enc_test[i,:,3] > 1]
        nodes = nodes[np.argsort(nodes[:, 1])]
        # Copy the last node's value to balance the number of leaves and nodes
        nodes = np.append(nodes, nodes[-1].reshape(1, -1), axis=0)
        # Pad nodes to ensure size of 500
        nodes = np.pad(nodes, [(0, (500 - nodes.shape[0])), (0, 0)], mode='constant')
        
        # Stack the leaves and nodes arrays together along axis 2 (creating 2 channels)
        enc_pad_test.append(np.stack((leaves, nodes), axis=2))
    
    # Convert lists to numpy arrays and return the padded training and test data
    return np.array(enc_pad), np.array(enc_pad_test)


#Change encoding to order by root age and pad with 0s
encoding_pad, encoding_pad_test = encode_pad_0s_rootage(encoding, encoding_test)

In [91]:
# Creation of the Network Model: model definition
def build_model():
    # Initialize the Sequential model
    model = Sequential()
    
    # First convolutional layer: 
    # - Filters: 32 
    # - Kernel size: (1, 20), sliding across the second dimension of the input 
    # - Input shape: (500, 20, 2) where 500 is the number of tree leaves/nodes, 20 is the feature size, and 2 is the number of channels (leaves and nodes)
    # - Activation function: ELU (Exponential Linear Unit)
    # - Groups: 2 to apply separate convolutions for the two channels (leaves and nodes)
    model.add(Conv2D(filters=32, use_bias=False, kernel_size=(1, 20), input_shape=(500, 20, 2), activation='elu', groups=2))
    
    # Apply batch normalization to stabilize and speed up the training process
    model.add(BatchNormalization())
    
    # Second convolutional layer: 
    # - Filters: 32
    # - Kernel size: (1, 1) to process each feature independently
    # - Activation function: ELU
    model.add(Conv2D(filters=32, use_bias=False, kernel_size=(1, 1), activation='elu'))
    
    # Apply batch normalization again
    model.add(BatchNormalization())
    
    # Third convolutional layer: 
    # - Filters: 32
    # - Kernel size: (1, 1) for further feature processing
    # - Activation function: ELU
    model.add(Conv2D(filters=32, use_bias=False, kernel_size=(1, 1), activation='elu'))
    
    # Apply batch normalization for the final time before flattening
    model.add(BatchNormalization())
    
    # Flatten the 2D feature maps from the convolutional layers into a 1D vector, 
    # which will be passed to the fully connected (dense) layers
    model.add(GlobalAveragePooling2D())
    
    # Fully connected (FFNN) part:
    # Dense layers with decreasing number of units, all using ELU activation:
    model.add(Dense(64, activation='elu'))   # First dense layer with 64 units
    model.add(Dense(32, activation='elu'))   # Second dense layer with 32 units
    model.add(Dense(16, activation='elu'))   # Third dense layer with 16 units
    model.add(Dense(8, activation='elu'))    # Fourth dense layer with 8 units
    
    # Output layer: 
    # - 4 output neurons, corresponding to the 4 target parameters to predict
    # - Activation function: ELU
    model.add(Dense(4, activation='elu'))
    
    # Show the summary of the model structure (number of layers, shapes of outputs, etc.)
    model.summary()

    # Return the constructed model
    return model


In [92]:
from keras import losses

# Initialize the model using the build_model function that was previously defined
estimator = build_model()

# Compile the model:
# - Loss function: 'mae' (mean absolute error) is used to measure the error in terms of percentage for regression problems
# - Optimizer: 'Adam' is used to minimize the loss function efficiently
# - Metrics: 'mean_absolute_percentage_error' (MAPE) is used to track the model's performance during training
estimator.compile(loss='mae', optimizer = 'Adam', metrics=[losses.mean_absolute_percentage_error])

# Early stopping callback to prevent overfitting:
# - monitor: monitor the validation loss during training
# - patience: stop training if the validation loss doesn't improve for 100 consecutive epochs
# - mode: 'min' indicates that training will stop when the validation loss reaches its minimum
# - restore_best_weights: restore the weights from the best epoch (the one with the lowest validation loss)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, mode='min', restore_best_weights=True)

# Custom callback to display training progress:
# - Print a dot for every epoch (or newline every 100 epochs) to indicate progress in training
class PrintD(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0:  # Print a newline every 100 epochs
            print('')
        print('.', end='')  # Print a dot to indicate progress during each epoch

# Set the maximum number of epochs (iterations over the entire dataset)
EPOCHS = 1000

# Train the model using the `fit` method:
# - encoding_pad: The padded training data (inputs)
# - Y: The target values (outputs)
# - verbose: set to 1 to print progress during training
# - epochs: The number of times to iterate over the entire dataset
# - validation_split: the fraction of data to use for validation (used to monitor validation loss)
# - batch_size: the number of samples per gradient update
# - callbacks: list of callbacks to be used during training (early stopping and progress display)
history = estimator.fit(encoding_pad, Y, verbose=1, epochs=EPOCHS, validation_split=valid_frac, batch_size=1024, callbacks=[early_stop, PrintD()])

# Save the model architecture to a JSON file:
# - The model structure (architecture) is saved as a JSON string
from keras.models import model_from_json
model = estimator.to_json()
with open('./Trained_Models/Trained_2Generation_BiSSE.json', 'w') as json_file:
    json_file.write(model)

# Save the model weights to an H5 file:
# - The weights (learned parameters) of the trained model are saved to a file
estimator.save_weights('./Trained_Models/Trained_2Generation_BiSSE.h5')

# Print a confirmation message when the model and weights are saved
print('model saved!')

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_33 (Conv2D)          (None, 500, 1, 32)        640       
                                                                 
 batch_normalization_33 (Ba  (None, 500, 1, 32)        128       
 tchNormalization)                                               
                                                                 
 conv2d_34 (Conv2D)          (None, 500, 1, 32)        1024      
                                                                 
 batch_normalization_34 (Ba  (None, 500, 1, 32)        128       
 tchNormalization)                                               
                                                                 
 conv2d_35 (Conv2D)          (None, 500, 1, 32)        1024      
                                                                 
 batch_normalization_35 (Ba  (None, 500, 1, 32)      

In [93]:
#load the model
from keras.models import model_from_json
json_file = open('./Trained_Models/Trained_2Generation_BiSSE.json', 'r')
model = json_file.read()
json_file.close()
estimator = model_from_json(model)
#load weights
estimator.load_weights('./Trained_Models/Trained_2Generation_BiSSE.h5')
print('model loaded!')

# predict values for the test set
predicted_test = pd.DataFrame(estimator.predict(encoding_pad_test))
predicted_test.columns = Y_test.columns # rename correctly the columns
predicted_test.index = Y_test.index # rename indexes for correspondence

model loaded!
1/1 [==============================] - 0s 33ms/step


In [94]:
####Save the target and predicted values in dataframes

#Target
Y_test_rescaled=pd.DataFrame(Y_test)
Y_test_rescaled['lambda1_rescaled'] = Y_test['lambda1_rescaled']/param_test['resc_factor']
Y_test_rescaled['lambda2_rescaled'] = Y_test['lambda2_rescaled']/param_test['resc_factor']
Y_test_rescaled['mu1_rescaled'] = Y_test['turnover']*Y_test['lambda1_rescaled']
Y_test_rescaled['mu2_rescaled'] = Y_test['turnover']*Y_test['lambda2_rescaled']
Y_test_rescaled['q01_rescaled'] = Y_test['q01_rescaled']/param_test['resc_factor']
Y_test_rescaled.to_csv('./Predictions/BiSSE_target.csv', header=True)

#Predicted
predicted_test['lambda1_rescaled'] = predicted_test['lambda1_rescaled']/param_test['resc_factor']
predicted_test['lambda2_rescaled'] = predicted_test['lambda2_rescaled']/param_test['resc_factor']
predicted_test['mu1_rescaled'] = predicted_test['turnover']*predicted_test['lambda1_rescaled']
predicted_test['mu2_rescaled'] = predicted_test['turnover']*predicted_test['lambda2_rescaled']
predicted_test['q01_rescaled'] = predicted_test['q01_rescaled']/param_test['resc_factor']
predicted_test.to_csv('./Predictions/2Generation_BiSSE_predicted.csv', header=True)

In [95]:
#Now I will remove the 2nd generation context to compare the networks
encoding_pad = encoding_pad[:,:,[0,1,2,3,4,5,6,7,8,9,19],:]
encoding_pad_test = encoding_pad_test[:,:,[0,1,2,3,4,5,6,7,8,9,19],:]

In [96]:
def build_model():
    # Initialize the Sequential model
    model = Sequential()
    
    # First convolutional layer: 
    # - Filters: 32 
    # - Kernel size: (1, 11), sliding across the second dimension of the input 
    # - Input shape: (500, 11, 2) where 500 is the number of tree leaves/nodes, 11 is the feature size, and 2 is the number of channels (leaves and nodes)
    # - Activation function: ELU (Exponential Linear Unit)
    # - Groups: 2 to apply separate convolutions for the two channels (leaves and nodes)
    model.add(Conv2D(filters=32, use_bias=False, kernel_size=(1, 11), input_shape=(500, 11, 2), activation='elu', groups=2))
    
    # Apply batch normalization to stabilize and speed up the training process
    model.add(BatchNormalization())
    
    # Second convolutional layer: 
    # - Filters: 32
    # - Kernel size: (1, 1) to process each feature independently
    # - Activation function: ELU
    model.add(Conv2D(filters=32, use_bias=False, kernel_size=(1, 1), activation='elu'))
    
    # Apply batch normalization again
    model.add(BatchNormalization())
    
    # Third convolutional layer: 
    # - Filters: 32
    # - Kernel size: (1, 1) for further feature processing
    # - Activation function: ELU
    model.add(Conv2D(filters=32, use_bias=False, kernel_size=(1, 1), activation='elu'))
    
    # Apply batch normalization for the final time before flattening
    model.add(BatchNormalization())
    
    # Flatten the 2D feature maps from the convolutional layers into a 1D vector, 
    # which will be passed to the fully connected (dense) layers
    model.add(GlobalAveragePooling2D())
    
    # Fully connected (FFNN) part:
    # Dense layers with decreasing number of units, all using ELU activation:
    model.add(Dense(64, activation='elu'))   # First dense layer with 64 units
    model.add(Dense(32, activation='elu'))   # Second dense layer with 32 units
    model.add(Dense(16, activation='elu'))   # Third dense layer with 16 units
    model.add(Dense(8, activation='elu'))    # Fourth dense layer with 8 units
    
    # Output layer: 
    # - 4 output neurons, corresponding to the 4 target parameters to predict
    # - Activation function: ELU
    model.add(Dense(4, activation='elu'))
    
    # Show the summary of the model structure (number of layers, shapes of outputs, etc.)
    model.summary()

    # Return the constructed model
    return model

In [97]:
from keras import losses

# Initialize the model using the build_model function that was previously defined
estimator = build_model()

# Compile the model:
# - Loss function: 'mae' (mean absolute error) is used to measure the error in terms of percentage for regression problems
# - Optimizer: 'Adam' is used to minimize the loss function efficiently
# - Metrics: 'mean_absolute_percentage_error' (MAPE) is used to track the model's performance during training
estimator.compile(loss='mae', optimizer = 'Adam', metrics=[losses.mean_absolute_percentage_error])

# Early stopping callback to prevent overfitting:
# - monitor: monitor the validation loss during training
# - patience: stop training if the validation loss doesn't improve for 100 consecutive epochs
# - mode: 'min' indicates that training will stop when the validation loss reaches its minimum
# - restore_best_weights: restore the weights from the best epoch (the one with the lowest validation loss)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, mode='min', restore_best_weights=True)

# Custom callback to display training progress:
# - Print a dot for every epoch (or newline every 100 epochs) to indicate progress in training
class PrintD(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0:  # Print a newline every 100 epochs
            print('')
        print('.', end='')  # Print a dot to indicate progress during each epoch

# Set the maximum number of epochs (iterations over the entire dataset)
EPOCHS = 1000

# Train the model using the `fit` method:
# - encoding_pad: The padded training data (inputs)
# - Y: The target values (outputs)
# - verbose: set to 1 to print progress during training
# - epochs: The number of times to iterate over the entire dataset
# - validation_split: the fraction of data to use for validation (used to monitor validation loss)
# - batch_size: the number of samples per gradient update
# - callbacks: list of callbacks to be used during training (early stopping and progress display)
history = estimator.fit(encoding_pad, Y, verbose=1, epochs=EPOCHS, validation_split=valid_frac, batch_size=1024, callbacks=[early_stop, PrintD()])

# Save the model architecture to a JSON file:
# - The model structure (architecture) is saved as a JSON string
from keras.models import model_from_json
model = estimator.to_json()
with open('./Trained_Models/Trained_1Generation_BiSSE.json', 'w') as json_file:
    json_file.write(model)

# Save the model weights to an H5 file:
# - The weights (learned parameters) of the trained model are saved to a file
estimator.save_weights('./Trained_Models/Trained_1Generation_BiSSE.h5')

# Print a confirmation message when the model and weights are saved
print('model saved!')

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_36 (Conv2D)          (None, 500, 1, 32)        352       
                                                                 
 batch_normalization_36 (Ba  (None, 500, 1, 32)        128       
 tchNormalization)                                               
                                                                 
 conv2d_37 (Conv2D)          (None, 500, 1, 32)        1024      
                                                                 
 batch_normalization_37 (Ba  (None, 500, 1, 32)        128       
 tchNormalization)                                               
                                                                 
 conv2d_38 (Conv2D)          (None, 500, 1, 32)        1024      
                                                                 
 batch_normalization_38 (Ba  (None, 500, 1, 32)      

In [98]:
#load the model
from keras.models import model_from_json
json_file = open('./Trained_Models/Trained_1Generation_BiSSE.json', 'r')
model = json_file.read()
json_file.close()
estimator = model_from_json(model)
#load weights
estimator.load_weights('./Trained_Models/Trained_1Generation_BiSSE.h5')
print('model loaded!')

# predict values for the test set
predicted_test = pd.DataFrame(estimator.predict(encoding_pad_test))
predicted_test.columns = Y_test.columns # rename correctly the columns
predicted_test.index = Y_test.index # rename indexes for correspondence

model loaded!
1/1 [==============================] - 0s 36ms/step


In [99]:
####Save the target and predicted values in dataframes

#Target
Y_test_rescaled=pd.DataFrame(Y_test)
Y_test_rescaled['lambda1_rescaled'] = Y_test['lambda1_rescaled']/param_test['resc_factor']
Y_test_rescaled['lambda2_rescaled'] = Y_test['lambda2_rescaled']/param_test['resc_factor']
Y_test_rescaled['mu1_rescaled'] = Y_test['turnover']*Y_test['lambda1_rescaled']
Y_test_rescaled['mu2_rescaled'] = Y_test['turnover']*Y_test['lambda2_rescaled']
Y_test_rescaled['q01_rescaled'] = Y_test['q01_rescaled']/param_test['resc_factor']
Y_test_rescaled.to_csv('./Predictions/BiSSE_target.csv', header=True)

#Predicted
predicted_test['lambda1_rescaled'] = predicted_test['lambda1_rescaled']/param_test['resc_factor']
predicted_test['lambda2_rescaled'] = predicted_test['lambda2_rescaled']/param_test['resc_factor']
predicted_test['mu1_rescaled'] = predicted_test['turnover']*predicted_test['lambda1_rescaled']
predicted_test['mu2_rescaled'] = predicted_test['turnover']*predicted_test['lambda2_rescaled']
predicted_test['q01_rescaled'] = predicted_test['q01_rescaled']/param_test['resc_factor']
predicted_test.to_csv('./Predictions/1Generation_BiSSE_predicted.csv', header=True)

In [100]:
#Now I will remove all context to compare the networks
encoding_pad = encoding_pad[:,:,[0,1,2,3,4,10],:]
encoding_pad_test = encoding_pad_test[:,:,[0,1,2,3,4,10],:]

In [101]:
def build_model():
    # Initialize the Sequential model
    model = Sequential()
    
    # First convolutional layer: 
    # - Filters: 32 
    # - Kernel size: (1, 6), sliding across the second dimension of the input 
    # - Input shape: (500, 6, 2) where 500 is the number of tree leaves/nodes, 6 is the feature size, and 2 is the number of channels (leaves and nodes)
    # - Activation function: ELU (Exponential Linear Unit)
    # - Groups: 2 to apply separate convolutions for the two channels (leaves and nodes)
    model.add(Conv2D(filters=32, use_bias=False, kernel_size=(1, 6), input_shape=(500, 6, 2), activation='elu', groups=2))
    
    # Apply batch normalization to stabilize and speed up the training process
    model.add(BatchNormalization())
    
    # Second convolutional layer: 
    # - Filters: 32
    # - Kernel size: (1, 1) to process each feature independently
    # - Activation function: ELU
    model.add(Conv2D(filters=32, use_bias=False, kernel_size=(1, 1), activation='elu'))
    
    # Apply batch normalization again
    model.add(BatchNormalization())
    
    # Third convolutional layer: 
    # - Filters: 32
    # - Kernel size: (1, 1) for further feature processing
    # - Activation function: ELU
    model.add(Conv2D(filters=32, use_bias=False, kernel_size=(1, 1), activation='elu'))
    
    # Apply batch normalization for the final time before flattening
    model.add(BatchNormalization())
    
    # Flatten the 2D feature maps from the convolutional layers into a 1D vector, 
    # which will be passed to the fully connected (dense) layers
    model.add(GlobalAveragePooling2D())
    
    # Fully connected (FFNN) part:
    # Dense layers with decreasing number of units, all using ELU activation:
    model.add(Dense(64, activation='elu'))   # First dense layer with 64 units
    model.add(Dense(32, activation='elu'))   # Second dense layer with 32 units
    model.add(Dense(16, activation='elu'))   # Third dense layer with 16 units
    model.add(Dense(8, activation='elu'))    # Fourth dense layer with 8 units
    
    # Output layer: 
    # - 4 output neurons, corresponding to the 4 target parameters to predict
    # - Activation function: ELU
    model.add(Dense(4, activation='elu'))
    
    # Show the summary of the model structure (number of layers, shapes of outputs, etc.)
    model.summary()

    # Return the constructed model
    return model

In [102]:
from keras import losses

# Initialize the model using the build_model function that was previously defined
estimator = build_model()

# Compile the model:
# - Loss function: 'mae' (mean absolute error) is used to measure the error in terms of percentage for regression problems
# - Optimizer: 'Adam' is used to minimize the loss function efficiently
# - Metrics: 'mean_absolute_percentage_error' (MAPE) is used to track the model's performance during training
estimator.compile(loss='mae', optimizer = 'Adam', metrics=[losses.mean_absolute_percentage_error])

# Early stopping callback to prevent overfitting:
# - monitor: monitor the validation loss during training
# - patience: stop training if the validation loss doesn't improve for 100 consecutive epochs
# - mode: 'min' indicates that training will stop when the validation loss reaches its minimum
# - restore_best_weights: restore the weights from the best epoch (the one with the lowest validation loss)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, mode='min', restore_best_weights=True)

# Custom callback to display training progress:
# - Print a dot for every epoch (or newline every 100 epochs) to indicate progress in training
class PrintD(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0:  # Print a newline every 100 epochs
            print('')
        print('.', end='')  # Print a dot to indicate progress during each epoch

# Set the maximum number of epochs (iterations over the entire dataset)
EPOCHS = 1000

# Train the model using the `fit` method:
# - encoding_pad: The padded training data (inputs)
# - Y: The target values (outputs)
# - verbose: set to 1 to print progress during training
# - epochs: The number of times to iterate over the entire dataset
# - validation_split: the fraction of data to use for validation (used to monitor validation loss)
# - batch_size: the number of samples per gradient update
# - callbacks: list of callbacks to be used during training (early stopping and progress display)
history = estimator.fit(encoding_pad, Y, verbose=1, epochs=EPOCHS, validation_split=valid_frac, batch_size=1024, callbacks=[early_stop, PrintD()])

# Save the model architecture to a JSON file:
# - The model structure (architecture) is saved as a JSON string
from keras.models import model_from_json
model = estimator.to_json()
with open('./Trained_Models/Trained_NoContext_BiSSE.json', 'w') as json_file:
    json_file.write(model)

# Save the model weights to an H5 file:
# - The weights (learned parameters) of the trained model are saved to a file
estimator.save_weights('./Trained_Models/Trained_NoContext_BiSSE.h5')

# Print a confirmation message when the model and weights are saved
print('model saved!')

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_39 (Conv2D)          (None, 500, 1, 32)        192       
                                                                 
 batch_normalization_39 (Ba  (None, 500, 1, 32)        128       
 tchNormalization)                                               
                                                                 
 conv2d_40 (Conv2D)          (None, 500, 1, 32)        1024      
                                                                 
 batch_normalization_40 (Ba  (None, 500, 1, 32)        128       
 tchNormalization)                                               
                                                                 
 conv2d_41 (Conv2D)          (None, 500, 1, 32)        1024      
                                                                 
 batch_normalization_41 (Ba  (None, 500, 1, 32)      

In [103]:
#load the model
from keras.models import model_from_json
json_file = open('./Trained_Models/Trained_NoContext_BiSSE.json', 'r')
model = json_file.read()
json_file.close()
estimator = model_from_json(model)
#load weights
estimator.load_weights('./Trained_Models/Trained_NoContext_BiSSE.h5')
print('model loaded!')

# predict values for the test set
predicted_test = pd.DataFrame(estimator.predict(encoding_pad_test))
predicted_test.columns = Y_test.columns # rename correctly the columns
predicted_test.index = Y_test.index # rename indexes for correspondence

model loaded!
1/1 [==============================] - 0s 36ms/step


In [104]:
####Save the target and predicted values in dataframes

#Target
Y_test_rescaled=pd.DataFrame(Y_test)
Y_test_rescaled['lambda1_rescaled'] = Y_test['lambda1_rescaled']/param_test['resc_factor']
Y_test_rescaled['lambda2_rescaled'] = Y_test['lambda2_rescaled']/param_test['resc_factor']
Y_test_rescaled['mu1_rescaled'] = Y_test['turnover']*Y_test['lambda1_rescaled']
Y_test_rescaled['mu2_rescaled'] = Y_test['turnover']*Y_test['lambda2_rescaled']
Y_test_rescaled['q01_rescaled'] = Y_test['q01_rescaled']/param_test['resc_factor']
Y_test_rescaled.to_csv('./Predictions/BiSSE_target.csv', header=True)

#Predicted
predicted_test['lambda1_rescaled'] = predicted_test['lambda1_rescaled']/param_test['resc_factor']
predicted_test['lambda2_rescaled'] = predicted_test['lambda2_rescaled']/param_test['resc_factor']
predicted_test['mu1_rescaled'] = predicted_test['turnover']*predicted_test['lambda1_rescaled']
predicted_test['mu2_rescaled'] = predicted_test['turnover']*predicted_test['lambda2_rescaled']
predicted_test['q01_rescaled'] = predicted_test['q01_rescaled']/param_test['resc_factor']
predicted_test.to_csv('./Predictions/NoContext_BiSSE_predicted.csv', header=True)